In [232]:
import pandas as pd
import numpy as np
import os
import datetime

In [233]:
def edit_column_date(frame,index):
    #Edits the date format of columns of dataframes
    #index: index of the first column of dates + 1
    i = 0
    for col in frame:
        i += 1
        if i >= index:
            new_d = date_format(col)
            frame = frame.rename(columns={col : new_d})
    return frame

In [234]:
def sort_dates(frame,index):
    #Sorts the columns by date of a frame with many nonconsecutive dates (several factors per date)
    Beg = list(frame.columns[:index]) #First four entries
    End = list(np.sort(np.array(frame.columns[index:]))) #Every Date Sorted
    cols = list(Beg + End) #Ordered Columns

    frame = frame[cols]
    return frame

In [235]:
def date_format(date):
    d = datetime.datetime.strptime(date, '%Y-%m-%d')
    return datetime.date.strftime(d, "%m/%d/%y")

In [236]:
pd.set_option("display.max_rows", 25, "display.max_columns", 25)

In [237]:
#Loading in mobility data
DL_us_m50 = pd.read_csv('../../../../data/us/mobility/DL-us-m50.csv', encoding='latin1')
DL_us_m50_index = pd.read_csv('../../../../data/us/mobility/DL-us-m50_index.csv', encoding='latin1')
DL_us_samples = pd.read_csv('../../../../data/us/mobility/DL-us-samples.csv')

In [238]:
#Cleaning the datasets
DL_us_m50 = edit_column_date(DL_us_m50,6)
DL_us_m50_index = edit_column_date(DL_us_m50_index,6)
DL_us_samples = edit_column_date(DL_us_samples,6)

DL_us_m50 = DL_us_m50.drop(columns=['country_code','admin_level','admin1','admin2'])
DL_us_m50_index = DL_us_m50_index.drop(columns=['country_code','admin_level','admin1','admin2'])
DL_us_samples = DL_us_samples.drop(columns=['country_code','admin_level','admin1','admin2'])

In [239]:
#Separating data into county info

DL_us_m50_County = DL_us_m50[DL_us_m50.fips >= 1000]
DL_us_m50_index_County = DL_us_m50_index[DL_us_m50_index.fips >= 1000]
DL_us_samples_County = DL_us_samples[DL_us_samples.fips >= 1000]

In [240]:
#merging the 3 datasets together
Mobility_County = pd.merge(DL_us_m50_County, DL_us_m50_index_County, \
                    left_on='fips', right_on='fips', suffixes=('_M_m50', ''), sort=True)
Mobility_County = pd.merge(Mobility_County, DL_us_samples_County, \
                    left_on='fips', right_on='fips', suffixes=('_M_idx', '_M_samples'), sort=True)
Mobility_County = Mobility_County[Mobility_County.fips >= -1]
Mobility_County.columns = Mobility_County.columns.str.replace('fips','FIPS')
#saving datasets with 3 values not consecutive and then consecutive
Mobility_County_Nonconsecutive = Mobility_County
Mobility_County_Consecutive = sort_dates(Mobility_County,1)
#MAking FIPS the main index
Mobility_County_Consecutive = Mobility_County_Consecutive.set_index('FIPS')
Mobility_County_Nonconsecutive = Mobility_County_Nonconsecutive.set_index('FIPS')

Mobility_County_Consecutive.to_csv('Mobility_County_Consecutive.csv')
Mobility_County_Nonconsecutive.to_csv('Mobility_County_Nonconsecutive.csv')

In [241]:
#New Google Mobility Data, must be processed

google_mobility = pd.read_csv('../../../../data/google_mobility/mobility_report_US.csv', encoding='latin1')
#Taking only county data
google_mobility_county = google_mobility[google_mobility['Region'] != 'Total']

#Key to map counties to FIPS, and states to state abbreviations
Key =  pd.read_csv('Key.csv').sort_values(by=['FIPS'])
State_Abv = pd.read_csv('../State_Abbrev.csv')
State_Abv = np.array(State_Abv)
#Dictionary from state names to state initials
State_Dict = dict((rows[0],rows[2]) for rows in State_Abv)

#Changing the state column of google mobility to its abbreviation code
google_mobility_county = google_mobility_county.replace({'State': State_Dict})

#Creating a location column, to make the google mobility locations unique
google_mobility_county['loc'] = google_mobility_county.Region.astype(str).str.cat(google_mobility_county.State.astype(str), sep=', ')
Key['loc'] = Key.COUNTY.astype(str).str.cat(Key.ST.astype(str), sep=', ')

#New google county mobility data, with fips codes attached
google_county = pd.merge(google_mobility_county, Key, \
                    left_on='loc', right_on='loc', sort=True)
#removing unecessary columns
google_county = google_county.drop(columns=['State','Region','ST','COUNTY','loc'])
google_county.to_csv('google_neighbors.csv')

FileNotFoundError: [Errno 2] File b'../../../../data/google_mobility/mobility_report_US.csv' does not exist: b'../../../../data/google_mobility/mobility_report_US.csv'

In [218]:
#Splitting up this google county into its components to rejoin it later
google_residential = google_county.pivot(index='FIPS', columns='Date', values=['Residential'])
google_residential.to_csv('google_residential.csv')
#Reading in split up component and the resetting the header values
google_residential = pd.read_csv('google_residential.csv',header=1).iloc[1:].rename(columns={'Date':'FIPS'})
google_residential = edit_column_date(google_residential,2)

google_workplaces = google_county.pivot(index='FIPS', columns='Date', values=['Workplaces'])
google_workplaces.to_csv('google_workplaces.csv')
google_workplaces = pd.read_csv('google_workplaces.csv',header=1).iloc[1:].rename(columns={'Date':'FIPS'})
google_workplaces = edit_column_date(google_workplaces,2)

google_transit = google_county.pivot(index='FIPS', columns='Date', values=['Transit stations'])
google_transit.to_csv('google_transit.csv')
google_transit = pd.read_csv('google_transit.csv',header=1).iloc[1:].rename(columns={'Date':'FIPS'})
google_transit = edit_column_date(google_transit,2)

google_parks = google_county.pivot(index='FIPS', columns='Date', values=['Parks'])
google_parks.to_csv('google_parks.csv')
google_parks = pd.read_csv('google_parks.csv',header=1).iloc[1:].rename(columns={'Date':'FIPS'})
google_parks = edit_column_date(google_parks,2)

google_grocery = google_county.pivot(index='FIPS', columns='Date', values=['Grocery & pharmacy'])
google_grocery.to_csv('google_grocery.csv')
google_grocery = pd.read_csv('google_grocery.csv',header=1).iloc[1:].rename(columns={'Date':'FIPS'})
google_grocery = edit_column_date(google_grocery,2)

google_retail = google_county.pivot(index='FIPS', columns='Date', values=['Retail & recreation'])
google_retail.to_csv('google_retail.csv')
google_retail = pd.read_csv('google_retail.csv',header=1).iloc[1:].rename(columns={'Date':'FIPS'})
google_retail = edit_column_date(google_retail,2)

#Merging the data back together
google_county = pd.merge(google_residential, google_workplaces, \
                    left_on='FIPS', right_on='FIPS', suffixes=('_residential', ''))
google_county = pd.merge(google_county, google_transit, \
                    left_on='FIPS', right_on='FIPS', suffixes=('_workplaces', ''))
google_county = pd.merge(google_county, google_parks, \
                    left_on='FIPS', right_on='FIPS', suffixes=('_transit', ''))
google_county = pd.merge(google_county, google_grocery, \
                    left_on='FIPS', right_on='FIPS', suffixes=('_parks', ''))
google_county = pd.merge(google_county, google_retail, \
                    left_on='FIPS', right_on='FIPS', suffixes=('_grocery', 'retail'))


In [219]:
#saving google dataset with each component either not consecutive and then consecutive
google_county_Nonconsecutive = google_county
google_county_Consecutive = sort_dates(google_county,1)

#MAking FIPS the main index
google_county_Consecutive = google_county_Consecutive.set_index('FIPS')
google_county_Nonconsecutive = google_county_Nonconsecutive.set_index('FIPS')

google_county_Consecutive.to_csv('google_county_Consecutive.csv')
google_county_Nonconsecutive.to_csv('google_county_Nonconsecutive.csv')

In [220]:
print('google_mobility: ' + str(len(google_mobility)))
print('Key: ' + str(len(Key)))
print('google_mobility_county: ' + str(len(google_mobility_county)))
print('google_county_Consecutive: ' + str(len(google_county_Consecutive)))


google_mobility: 8603
Key: 3246
google_mobility_county: 8450
google_county_Consecutive: 2766


In [221]:
print('DL_us_m50_County: ' + str(len(DL_us_m50_County)))
print('DL_us_m50_index_County: ' + str(len(DL_us_m50_index_County)))
print('DL_us_samples_County: ' + str(len(DL_us_samples_County)))
print('Mobility_County_Consecutive: ' + str(len(Mobility_County_Consecutive)))

DL_us_m50_County: 2670
DL_us_m50_index_County: 2670
DL_us_samples_County: 2670
Mobility_County_Consecutive: 2670


In [222]:
Mobility_County_Consecutive.head()

,03/01/20_M_idx,03/01/20_M_m50,03/01/20_M_samples,03/02/20_M_idx,03/02/20_M_m50,03/02/20_M_samples,03/03/20_M_idx,03/03/20_M_m50,03/03/20_M_samples,03/04/20_M_idx,03/04/20_M_m50,03/04/20_M_samples,...,04/12/20_M_idx,04/12/20_M_m50,04/12/20_M_samples,04/13/20_M_idx,04/13/20_M_m50,04/13/20_M_samples,04/14/20_M_idx,04/14/20_M_m50,04/14/20_M_samples,04/15/20_M_idx,04/15/20_M_m50,04/15/20_M_samples
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,
1001.0,49.0,7.194,1703.0,100.0,14.587,1829.0,95.0,13.865,1840.0,95.0,13.880,1900.0,...,1.0,0.209,1693.0,43.0,6.356,1627.0,36.0,5.348,1667.0,42.0,6.241,1720.0
1003.0,81.0,9.780,7067.0,100.0,12.042,7136.0,95.0,11.481,7220.0,90.0,10.879,7370.0,...,1.0,0.126,6534.0,48.0,5.877,6274.0,49.0,5.934,6338.0,52.0,6.382,6518.0
1005.0,90.0,8.348,546.0,107.0,10.004,569.0,100.0,9.267,545.0,70.0,6.517,578.0,...,13.0,1.215,511.0,50.0,4.716,504.0,51.0,4.762,501.0,54.0,5.049,532.0
1007.0,53.0,13.008,512.0,95.0,23.076,574.0,100.0,24.164,588.0,94.0,22.768,594.0,...,3.0,0.760,516.0,43.0,10.603,495.0,51.0,12.359,509.0,50.0,12.150,523.0
1009.0,68.0,15.963,1495.0,96.0,22.456,1608.0,100.0,23.222,1615.0,99.0,23.035,1627.0,...,2.0,0.681,1485.0,56.0,13.091,1404.0,59.0,13.888,1419.0,68.0,15.823,1498.0


In [223]:
google_county_Consecutive.head()

,03/29/20_grocery,03/29/20_parks,03/29/20_residential,03/29/20_transit,03/29/20_workplaces,03/29/20retail,04/05/20_grocery,04/05/20_parks,04/05/20_residential,04/05/20_transit,04/05/20_workplaces,04/05/20retail,04/11/20_grocery,04/11/20_parks,04/11/20_residential,04/11/20_transit,04/11/20_workplaces,04/11/20retail
FIPS,,,,,,,,,,,,,,,,,,
1001,-8.0,-14.0,16.0,NaN,-35.0,-42.0,-8.0,-14.0,17.0,NaN,-38.0,-53.0,16.0,-14.0,20.0,NaN,-30.0,-37.0
1003,-19.0,-27.0,9.0,-24.0,-32.0,-47.0,-27.0,-44.0,11.0,-39.0,-35.0,-55.0,-10.0,-42.0,11.0,-39.0,-28.0,-44.0
1005,-20.0,NaN,NaN,NaN,-23.0,-27.0,-20.0,NaN,NaN,NaN,-30.0,-40.0,-20.0,NaN,NaN,NaN,-31.0,-18.0
1007,-10.0,NaN,NaN,NaN,-32.0,-25.0,-15.0,NaN,NaN,NaN,-35.0,-42.0,-15.0,NaN,NaN,NaN,-19.0,-42.0
1009,-2.0,NaN,15.0,NaN,-29.0,-36.0,-17.0,NaN,14.0,NaN,-33.0,-49.0,18.0,NaN,20.0,NaN,-24.0,-20.0


In [224]:
#Taking Google Extracted Data
google_extrapolated = pd.read_csv('../../../../data/google_mobility/extrapolated_mobility_report_US.csv', encoding='latin1')
google_extrapolated = google_extrapolated.replace({'state': State_Dict})
#Fixing some row names so they can be processed to fips easily
google_extrapolated.loc[google_extrapolated['county'] == 'Anchorage', 'county'] = 'Anchorage Municipality'
google_extrapolated.loc[google_extrapolated['county'] == 'Baltimore', 'county'] = 'Baltimore County'
google_extrapolated.loc[google_extrapolated['county'] == 'DoÃ±a Ana County', 'county'] = 'Dona Ana County'
google_extrapolated.loc[google_extrapolated['county'] == 'Falls Church', 'county'] = 'Falls Church city'
google_extrapolated.loc[google_extrapolated['county'] == 'Harrisonburg', 'county'] = 'Harrisonburg city'
google_extrapolated.loc[google_extrapolated['county'] == 'Martinsville', 'county'] = 'Martinsville city'
google_extrapolated.loc[google_extrapolated['county'] == 'Danville', 'county'] = 'Danville city'
google_extrapolated.loc[google_extrapolated['county'] == 'Petersburg', 'county'] = 'Petersburg city'
google_extrapolated.loc[google_extrapolated['county'] == 'Waynesboro', 'county'] = 'Waynesboro city'
google_extrapolated.loc[google_extrapolated['county'] == 'Winchester', 'county'] = 'Winchester city'
google_extrapolated.loc[google_extrapolated['county'] == 'Richmond', 'county'] = 'Richmond city'
google_extrapolated.loc[google_extrapolated['county'] == 'St. Louis', 'county'] = 'St. Louis city'
google_extrapolated.loc[google_extrapolated['county'] == 'LaPorte County', 'county'] = 'Laporte County'
google_extrapolated.loc[google_extrapolated['county'] == 'LaSalle County', 'county'] = 'Lasalle County'
#Adding fips data to google using Key from fips to county names
google_extrapolated['loc'] = google_extrapolated.county.astype(str).str.cat(google_extrapolated.state.astype(str), sep=', ')
google_extrapolated = pd.merge(google_extrapolated, Key, left_on='loc', right_on='loc', sort=True)
google_extrapolated = google_extrapolated.drop(columns = ['state','county','change','changecalc','loc','ST','COUNTY']).sort_values(['FIPS', 'category']).reset_index(drop = True)

In [225]:
#List of categories included in mobility data
Cat_List = list(google_extrapolated.category.unique())
Cat_List

['Grocery & pharmacy',
 'Retail & recreation',
 'Workplace',
 'Parks',
 'Residential',
 'Transit stations']

In [226]:
def getval(fips, date, category, google_extrapolated):
    #Gets the value from google extrapolated based on the fips, date and mobility category
    df = google_extrapolated.copy()
    df = df[df['FIPS'] == fips]
    df = df[df['category'] == category]
    if date == '2020-02-29':
        print(fips)
    if len(df) == 0:
        return np.nan
    else:
        if list(np.isnan(df[date]))[0]:
            return np.nan
        else:
            return float(sum(list(df[date]))/len(list(df[date])))

In [227]:
#This is a list of all the counties and dates
neighborcounties = pd.read_csv('neighborcounties.csv', encoding='latin1', index_col=0)
County_List = list(neighborcounties.orgfips.unique())
Date_List = list(google_extrapolated.columns[1:-1])
#This creates a base dataframe that contains all pairs of FIPS codes with the valid dates given in google_extrapolated
CL, DL = pd.core.reshape.util.cartesian_product([County_List, Date_List])
#Making a frame for each county and category for each date
BaseFrame = pd.DataFrame(dict(FIPS=CL, Date=DL)).sort_values(['FIPS','Date']).reset_index(drop=True)
BaseFrame['Grocery & pharmacy'] = BaseFrame.apply(lambda x: getval(x['FIPS'], x['Date'], 'Grocery & pharmacy', google_extrapolated), axis = 1)
BaseFrame['Retail & recreation'] = BaseFrame.apply(lambda x: getval(x['FIPS'], x['Date'], 'Retail & recreation', google_extrapolated), axis = 1)
BaseFrame['Workplace'] = BaseFrame.apply(lambda x: getval(x['FIPS'], x['Date'], 'Workplace', google_extrapolated), axis = 1)
BaseFrame['Parks'] = BaseFrame.apply(lambda x: getval(x['FIPS'], x['Date'], 'Parks', google_extrapolated), axis = 1)
BaseFrame['Residential'] = BaseFrame.apply(lambda x: getval(x['FIPS'], x['Date'], 'Residential', google_extrapolated), axis = 1)
BaseFrame['Transit stations'] = BaseFrame.apply(lambda x: getval(x['FIPS'], x['Date'], 'Transit stations', google_extrapolated), axis = 1)

BaseFrame.to_csv('google_extrapolated_base.csv')

1001
1003
1005
1007
1009
1011
1013
1015
1017
1019
1021
1023
1025
1027
1029
1031
1033
1035
1037
1039
1041
1043
1045
1047
1049
1051
1053
1055
1057
1059
1061
1063
1065
1067
1069
1071
1073
1075
1077
1079
1081
1083
1085
1087
1089
1091
1093
1095
1097
1099
1101
1103
1105
1107
1109
1111
1113
1115
1117
1119
1121
1123
1125
1127
1129
1131
1133
2013
2016
2020
2050
2060
2068
2070
2090
2100
2105
2110
2122
2130
2150
2164
2170
2180
2185
2188
2195
2198
2220
2230
2240
2261
2270
2275
2282
2290
4001
4003
4005
4007
4009
4011
4012
4013
4015
4017
4019
4021
4023
4025
4027
5001
5003
5005
5007
5009
5011
5013
5015
5017
5019
5021
5023
5025
5027
5029
5031
5033
5035
5037
5039
5041
5043
5045
5047
5049
5051
5053
5055
5057
5059
5061
5063
5065
5067
5069
5071
5073
5075
5077
5079
5081
5083
5085
5087
5089
5091
5093
5095
5097
5099
5101
5103
5105
5107
5109
5111
5113
5115
5117
5119
5121
5123
5125
5127
5129
5131
5133
5135
5137
5139
5141
5143
5145
5147
5149
6001
6003
6005
6007
6009
6011
6013
6015
6017
6019
6021
6023
6025
6027


28043
28045
28047
28049
28051
28053
28055
28057
28059
28061
28063
28065
28067
28069
28071
28073
28075
28077
28079
28081
28083
28085
28087
28089
28091
28093
28095
28097
28099
28101
28103
28105
28107
28109
28111
28113
28115
28117
28119
28121
28123
28125
28127
28129
28131
28133
28135
28137
28139
28141
28143
28145
28147
28149
28151
28153
28155
28157
28159
28161
28163
29001
29003
29005
29007
29009
29011
29013
29015
29017
29019
29021
29023
29025
29027
29029
29031
29033
29035
29037
29039
29041
29043
29045
29047
29049
29051
29053
29055
29057
29059
29061
29063
29065
29067
29069
29071
29073
29075
29077
29079
29081
29083
29085
29087
29089
29091
29093
29095
29097
29099
29101
29103
29105
29107
29109
29111
29113
29115
29117
29119
29121
29123
29125
29127
29129
29131
29133
29135
29137
29139
29141
29143
29145
29147
29149
29151
29153
29155
29157
29159
29161
29163
29165
29167
29169
29171
29173
29175
29177
29179
29181
29183
29185
29186
29187
29189
29195
29197
29199
29201
29203
29205
29207
29209
29211
2921

49027
49029
49031
49033
49035
49037
49039
49041
49043
49045
49047
49049
49051
49053
49055
49057
50001
50003
50005
50007
50009
50011
50013
50015
50017
50019
50021
50023
50025
50027
51001
51003
51005
51007
51009
51011
51013
51015
51017
51019
51021
51023
51025
51027
51029
51031
51033
51035
51036
51037
51041
51043
51045
51047
51049
51051
51053
51057
51059
51061
51063
51065
51067
51069
51071
51073
51075
51077
51079
51081
51083
51085
51087
51089
51091
51093
51095
51097
51099
51101
51103
51105
51107
51109
51111
51113
51115
51117
51119
51121
51125
51127
51131
51133
51135
51137
51139
51141
51143
51145
51147
51149
51153
51155
51157
51159
51161
51163
51165
51167
51169
51171
51173
51175
51177
51179
51181
51183
51185
51187
51191
51193
51195
51197
51199
51510
51515
51520
51530
51540
51550
51570
51580
51590
51595
51600
51610
51620
51630
51640
51650
51660
51670
51678
51680
51683
51685
51690
51700
51710
51720
51730
51735
51740
51750
51760
51770
51775
51790
51800
51810
51820
51830
51840
53001
53003
5300

21001
21003
21005
21007
21009
21011
21013
21015
21017
21019
21021
21023
21025
21027
21029
21031
21033
21035
21037
21039
21041
21043
21045
21047
21049
21051
21053
21055
21057
21059
21061
21063
21065
21067
21069
21071
21073
21075
21077
21079
21081
21083
21085
21087
21089
21091
21093
21095
21097
21099
21101
21103
21105
21107
21109
21111
21113
21115
21117
21119
21121
21123
21125
21127
21129
21131
21133
21135
21137
21139
21141
21143
21145
21147
21149
21151
21153
21155
21157
21159
21161
21163
21165
21167
21169
21171
21173
21175
21177
21179
21181
21183
21185
21187
21189
21191
21193
21195
21197
21199
21201
21203
21205
21207
21209
21211
21213
21215
21217
21219
21221
21223
21225
21227
21229
21231
21233
21235
21237
21239
22001
22003
22005
22007
22009
22011
22013
22015
22017
22019
22021
22023
22025
22027
22029
22031
22033
22035
22037
22039
22041
22043
22045
22047
22049
22051
22053
22055
22057
22059
22061
22063
22065
22067
22069
22071
22073
22075
22077
22079
22081
22083
22085
22087
22089
22091
2209

45091
46003
46005
46007
46009
46011
46013
46015
46017
46019
46021
46023
46025
46027
46029
46031
46033
46035
46037
46039
46041
46043
46045
46047
46049
46051
46053
46055
46057
46059
46061
46063
46065
46067
46069
46071
46073
46075
46077
46079
46081
46083
46085
46087
46089
46091
46093
46095
46097
46099
46101
46103
46105
46107
46109
46111
46113
46115
46117
46119
46121
46123
46125
46127
46129
46135
46137
47001
47003
47005
47007
47009
47011
47013
47015
47017
47019
47021
47023
47025
47027
47029
47031
47033
47035
47037
47039
47041
47043
47045
47047
47049
47051
47053
47055
47057
47059
47061
47063
47065
47067
47069
47071
47073
47075
47077
47079
47081
47083
47085
47087
47089
47091
47093
47095
47097
47099
47101
47103
47105
47107
47109
47111
47113
47115
47117
47119
47121
47123
47125
47127
47129
47131
47133
47135
47137
47139
47141
47143
47145
47147
47149
47151
47153
47155
47157
47159
47161
47163
47165
47167
47169
47171
47173
47175
47177
47179
47181
47183
47185
47187
47189
48001
48003
48005
48007
4800

...

51089
51091
51093
51095
51097
51099
51101
51103
51105
51107
51109
51111
51113
51115
51117
51119
51121
51125
51127
51131
51133
51135
51137
51139
51141
51143
51145
51147
51149
51153
51155
51157
51159
51161
51163
51165
51167
51169
51171
51173
51175
51177
51179
51181
51183
51185
51187
51191
51193
51195
51197
51199
51510
51515
51520
51530
51540
51550
51570
51580
51590
51595
51600
51610
51620
51630
51640
51650
51660
51670
51678
51680
51683
51685
51690
51700
51710
51720
51730
51735
51740
51750
51760
51770
51775
51790
51800
51810
51820
51830
51840
53001
53003
53005
53007
53009
53011
53013
53015
53017
53019
53021
53023
53025
53027
53029
53031
53033
53035
53037
53039
53041
53043
53045
53047
53049
53051
53053
53055
53057
53059
53061
53063
53065
53067
53069
53071
53073
53075
53077
54001
54003
54005
54007
54009
54011
54013
54015
54017
54019
54021
54023
54025
54027
54029
54031
54033
54035
54037
54039
54041
54043
54045
54047
54049
54051
54053
54055
54057
54059
54061
54063
54065
54067
54069
54071
5407

21147
21149
21151
21153
21155
21157
21159
21161
21163
21165
21167
21169
21171
21173
21175
21177
21179
21181
21183
21185
21187
21189
21191
21193
21195
21197
21199
21201
21203
21205
21207
21209
21211
21213
21215
21217
21219
21221
21223
21225
21227
21229
21231
21233
21235
21237
21239
22001
22003
22005
22007
22009
22011
22013
22015
22017
22019
22021
22023
22025
22027
22029
22031
22033
22035
22037
22039
22041
22043
22045
22047
22049
22051
22053
22055
22057
22059
22061
22063
22065
22067
22069
22071
22073
22075
22077
22079
22081
22083
22085
22087
22089
22091
22093
22095
22097
22099
22101
22103
22105
22107
22109
22111
22113
22115
22117
22119
22121
22123
22125
22127
23001
23003
23005
23007
23009
23011
23013
23015
23017
23019
23021
23023
23025
23027
23029
23031
24001
24003
24005
24009
24011
24013
24015
24017
24019
24021
24023
24025
24027
24029
24031
24033
24035
24037
24039
24041
24043
24045
24047
24510
25001
25003
25005
25007
25009
25011
25013
25015
25017
25019
25021
25023
25025
25027
26001
2600

47011
47013
47015
47017
47019
47021
47023
47025
47027
47029
47031
47033
47035
47037
47039
47041
47043
47045
47047
47049
47051
47053
47055
47057
47059
47061
47063
47065
47067
47069
47071
47073
47075
47077
47079
47081
47083
47085
47087
47089
47091
47093
47095
47097
47099
47101
47103
47105
47107
47109
47111
47113
47115
47117
47119
47121
47123
47125
47127
47129
47131
47133
47135
47137
47139
47141
47143
47145
47147
47149
47151
47153
47155
47157
47159
47161
47163
47165
47167
47169
47171
47173
47175
47177
47179
47181
47183
47185
47187
47189
48001
48003
48005
48007
48009
48011
48013
48015
48017
48019
48021
48023
48025
48027
48029
48031
48033
48035
48037
48039
48041
48043
48045
48047
48049
48051
48053
48055
48057
48059
48061
48063
48065
48067
48069
48071
48073
48075
48077
48079
48081
48083
48085
48087
48089
48091
48093
48095
48097
48099
48101
48103
48105
48107
48109
48111
48113
48115
48117
48119
48121
48123
48125
48127
48129
48131
48133
48135
48137
48139
48141
48143
48145
48147
48149
48151
4815

17083
17085
17087
17089
17091
17093
17095
17097
17099
17101
17103
17105
17107
17109
17111
17113
17115
17117
17119
17121
17123
17125
17127
17129
17131
17133
17135
17137
17139
17141
17143
17145
17147
17149
17151
17153
17155
17157
17159
17161
17163
17165
17167
17169
17171
17173
17175
17177
17179
17181
17183
17185
17187
17189
17191
17193
17195
17197
17199
17201
17203
18001
18003
18005
18007
18009
18011
18013
18015
18017
18019
18021
18023
18025
18027
18029
18031
18033
18035
18037
18039
18041
18043
18045
18047
18049
18051
18053
18055
18057
18059
18061
18063
18065
18067
18069
18071
18073
18075
18077
18079
18081
18083
18085
18087
18089
18091
18093
18095
18097
18099
18101
18103
18105
18107
18109
18111
18113
18115
18117
18119
18121
18123
18125
18127
18129
18131
18133
18135
18137
18139
18141
18143
18145
18147
18149
18151
18153
18155
18157
18159
18161
18163
18165
18167
18169
18171
18173
18175
18177
18179
18181
18183
19001
19003
19005
19007
19009
19011
19013
19015
19017
19019
19021
19023
19025
1902

38027
38029
38031
38033
38035
38037
38039
38041
38043
38045
38047
38049
38051
38053
38055
38057
38059
38061
38063
38065
38067
38069
38071
38073
38075
38077
38079
38081
38083
38085
38087
38089
38091
38093
38095
38097
38099
38101
38103
38105
39001
39003
39005
39007
39009
39011
39013
39015
39017
39019
39021
39023
39025
39027
39029
39031
39033
39035
39037
39039
39041
39043
39045
39047
39049
39051
39053
39055
39057
39059
39061
39063
39065
39067
39069
39071
39073
39075
39077
39079
39081
39083
39085
39087
39089
39091
39093
39095
39097
39099
39101
39103
39105
39107
39109
39111
39113
39115
39117
39119
39121
39123
39125
39127
39129
39131
39133
39135
39137
39139
39141
39143
39145
39147
39149
39151
39153
39155
39157
39159
39161
39163
39165
39167
39169
39171
39173
39175
40001
40003
40005
40007
40009
40011
40013
40015
40017
40019
40021
40023
40025
40027
40029
40031
40033
40035
40037
40039
40041
40043
40045
40047
40049
40051
40053
40055
40057
40059
40061
40063
40065
40067
40069
40071
40073
40075
4007

In [231]:
BaseFrame.head(10030)

,FIPS,Date,Grocery & pharmacy,Retail & recreation,Workplace,Parks,Residential,Transit stations
0,1001,2020-02-29,10.146,22.367,5.419,NaN,NaN,NaN
1,1001,2020-03-01,13.120,15.777,2.815,NaN,NaN,NaN
2,1001,2020-03-02,3.976,15.392,4.764,NaN,NaN,NaN
3,1001,2020-03-03,8.413,22.508,3.780,NaN,NaN,NaN
4,1001,2020-03-04,-4.022,5.373,2.905,NaN,NaN,NaN
5,1001,2020-03-05,12.303,7.745,2.911,NaN,NaN,NaN
6,1001,2020-03-06,10.348,4.127,4.141,NaN,NaN,NaN
7,1001,2020-03-07,10.823,9.842,10.838,NaN,NaN,NaN
8,1001,2020-03-08,8.349,10.219,9.150,NaN,NaN,NaN
9,1001,2020-03-09,11.927,15.392,3.682,NaN,NaN,NaN
